**TRAIN PIPELINE**

In [1]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 100.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstal

In [2]:
!pip install -U spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 59.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.5
    Uninstalling spacy-3.8.5:
      Successfully uninstalled spacy-3.8.5


In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [4]:
import os
print(os.listdir('.'))

['__notebook__.ipynb']


In [5]:
!nvidia-smi

Thu May 22 16:11:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
cv_data = json.load(open("/kaggle/input/resumedataset/train_data.json"))

In [7]:
!python -m spacy init fill-config /kaggle/input/resumedataset/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot["entities"]
        ents = []
        entity_indices = []
        for start, end, label in annot:
          skip_entity = False
          for idx in range(start, end):
            if idx in entity_indices:
              print('TRUE',idx)
              skip_entity = True
              break
          if skip_entity == True:
            continue
          entity_indices = entity_indices + list(range(start, end))
          try:
            span = doc.char_span(start, end, label=label, alignment_mode="strict")
          except:
            continue
          if span is None:
            err_data = str([start, end]) + " " + str(text) + "\n"
            file.write(err_data)
          else:
            ents.append(span)
        try:
          doc.ents = ents
          db.add(doc)
        except:
          pass
    return db

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [10]:
len(train), len(test)

(140, 60)

In [11]:
file = open("error_data.txt", "w")
db = get_spacy_doc(file, train)
db.to_disk("train_data.spacy")

db = get_spacy_doc(file, test)
db.to_disk("test_data.spacy")

file.close()

 15%|█▌        | 21/140 [00:00<00:01, 88.80it/s]

TRUE 6682
TRUE 823
TRUE 21
TRUE 3981
TRUE 3803
TRUE 46
TRUE 1258
TRUE 13883
TRUE 11578
TRUE 3243
TRUE 667


 31%|███       | 43/140 [00:00<00:01, 93.10it/s]

TRUE 2128
TRUE 2076
TRUE 1865
TRUE 381
TRUE 983
TRUE 67
TRUE 3
TRUE 15
TRUE 4708
TRUE 15
TRUE 370
TRUE 13
TRUE 34


 45%|████▌     | 63/140 [00:00<00:00, 87.02it/s]

...

 40%|████      | 24/60 [00:00<00:00, 73.57it/s]

TRUE 1180
TRUE 13
TRUE 4231
TRUE 6861
TRUE 7777
TRUE 4774


 72%|███████▏  | 43/60 [00:00<00:00, 81.52it/s]

TRUE 2907
TRUE 10
TRUE 497
TRUE 819
TRUE 210
TRUE 203
TRUE 1106
TRUE 996
TRUE 43
TRUE 1514
TRUE 47
TRUE 15


100%|██████████| 60/60 [00:00<00:00, 92.02it/s] 


In [12]:
with open("/kaggle/working/config.cfg", "r") as f:
    config_text = f.read()
print(config_text)


[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = "pytorch"
seed = 0

[nlp]
lang = "en"
pipeline = ["transformer","ner"]
batch_size = 128
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}
vectors = {"@vectors":"spacy.Vectors.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0
pooling = {"@layers":"reduce_mean.v1"}
upstream = "*"

[components.transformer]
factory = "transformer"
max_batch_items = 4096
set_extra_annotations = {"@annotation_setters":"sp

**CUSTOM CONFIG**

In [13]:
config_text = config_text.replace("max_steps = 20000", "max_steps = 40000")
config_text = config_text.replace("patience = 1600", "patience = 3000")
config_text = config_text.replace("dropout = 0.1", "dropout = 0.05")
config_text = config_text.replace("use_averages = false", "use_averages = true")
config_text = config_text.replace("@schedules = \"warmup_linear.v1\"", "@schedules = \"warmup_cosine.v1\"")
config_text = config_text.replace("warmup_steps = 250", "warmup_steps = 500")
config_text = config_text.replace("total_steps = 20000", "total_steps = 40000")

In [14]:
print(config_text)

[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = "pytorch"
seed = 0

[nlp]
lang = "en"
pipeline = ["transformer","ner"]
batch_size = 128
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}
vectors = {"@vectors":"spacy.Vectors.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0
pooling = {"@layers":"reduce_mean.v1"}
upstream = "*"

[components.transformer]
factory = "transformer"
max_batch_items = 4096
set_extra_annotations = {"@annotation_setters":"sp

In [15]:
!python -m spacy train /kaggle/working/config.cfg \
--output /kaggle/working/output \
--paths.train /kaggle/working/train_data.spacy \
--paths.dev /kaggle/working/test_data.spacy \
--gpu-id 0


✔ Created output directory: /kaggle/working/output
ℹ Saving to output directory: /kaggle/working/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2025-05-22 16:12:22,707] [INFO] Set up nlp object from config
[2025-05-22 16:12:22,727] [INFO] Pipeline: ['transformer', 'ner']
[2025-05-22 16:12:22,729] [INFO] Created vocabulary
[2025-05-22 16:12:22,729] [INFO] Finished initializing nlp object
tokenizer_config.json: 100%|██████████████████| 25.0/25.0 [00:00<00:00, 185kB/s]
config.json: 100%|█████████████████████████████| 481/481 [00:00<00:00, 3.21MB/s]
vocab.json: 100%|████████████████████████████| 899k/899k [00:00<00:00, 5.62MB/s]
merges.txt: 100%|████████████████████████████| 456k/456k [00:00<00:00, 19.5MB/s]
tokenizer.json: 100%|██████████████████████| 1.36M/1.36M [00:00<00:00, 20.6MB/s]
2025-05-22 16:12:30.540230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register fa